In [1]:
import time
import itertools
import pandas as pd
import numpy as np
import seaborn
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
data=pd.read_csv("Telco-Customer-Churn.csv")
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [4]:
#CustomerID feature has no use so dropped it.
data.drop('customerID', axis='columns', inplace=True)

In [5]:
#Since TotalCharges is object Type (It should have come under quantitative feature), I would change it to numeric data type
data.TotalCharges=pd.to_numeric(data.TotalCharges,errors='coerce')

In [6]:
data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [7]:
data.describe(include=['object']).T

,count,unique,top,freq
gender,7043,2,Male,3555
Partner,7043,2,No,3641
Dependents,7043,2,No,4933
PhoneService,7043,2,Yes,6361
MultipleLines,7043,3,No,3390
InternetService,7043,3,Fiber optic,3096
OnlineSecurity,7043,3,No,3498
OnlineBackup,7043,3,No,3088
DeviceProtection,7043,3,No,3095
TechSupport,7043,3,No,3473


In [8]:
data.isnull().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [9]:
# let us drop all rows with any missing value
data.dropna(inplace=True)

In [10]:
data.replace('No internet service', 'No', inplace=True)
data.replace('No phone service', 'No', inplace=True)

In [11]:
for col in data.columns:
    print(f'{col}: {data[col].unique()}')

gender: ['Female' 'Male']
SeniorCitizen: [0 1]
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
tenure: [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
PhoneService: ['No' 'Yes']
MultipleLines: ['No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes']
OnlineBackup: ['Yes' 'No']
DeviceProtection: ['No' 'Yes']
TechSupport: ['No' 'Yes']
StreamingTV: ['No' 'Yes']
StreamingMovies: ['No' 'Yes']
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
MonthlyCharges: [29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
TotalCharges: [  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]
Churn: ['No' 'Yes']


In [12]:
#Encode categorical features
cat_cols = data.select_dtypes(include=['object']).columns.drop(['Churn','InternetService', 'Contract', 'PaymentMethod'])
for col in cat_cols:
    data[col] = LabelEncoder().fit_transform(data[col])

In [13]:
data['Churn'] = data['Churn'].map({'Yes':1, 'No':0})

In [14]:
data = pd.get_dummies(data, columns=['InternetService', 'Contract', 'PaymentMethod']) # one hot encoding

In [15]:
# Convert all bools to integers
for col in data.select_dtypes(include='bool').columns:
    data[col] = data[col].astype(int)

In [16]:
for col in data.columns:
  print(f'{col}: {data[col].unique()}')

gender: [0 1]
SeniorCitizen: [0 1]
Partner: [1 0]
Dependents: [0 1]
tenure: [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
PhoneService: [0 1]
MultipleLines: [0 1]
OnlineSecurity: [0 1]
OnlineBackup: [1 0]
DeviceProtection: [0 1]
TechSupport: [0 1]
StreamingTV: [0 1]
StreamingMovies: [0 1]
PaperlessBilling: [1 0]
MonthlyCharges: [29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
TotalCharges: [  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]
Churn: [0 1]
InternetService_DSL: [1 0]
InternetService_Fiber optic: [0 1]
InternetService_No: [0 1]
Contract_Month-to-month: [1 0]
Contract_One year: [0 1]
Contract_Two year: [0 1]
PaymentMethod_Bank transfer (automatic): [0 1]
PaymentMethod_Credit card (automatic): [0 1]
PaymentMethod_Electronic check: [1 0]
PaymentMethod_Mailed check: [0 1]


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 27 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   gender                                   7032 non-null   int32  
 1   SeniorCitizen                            7032 non-null   int64  
 2   Partner                                  7032 non-null   int32  
 3   Dependents                               7032 non-null   int32  
 4   tenure                                   7032 non-null   int64  
 5   PhoneService                             7032 non-null   int32  
 6   MultipleLines                            7032 non-null   int32  
 7   OnlineSecurity                           7032 non-null   int32  
 8   OnlineBackup                             7032 non-null   int32  
 9   DeviceProtection                         7032 non-null   int32  
 10  TechSupport                              7032 non-nul

In [18]:
X = data.drop('Churn', axis=1).values
y = data['Churn'].values

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [20]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [21]:
X_train.shape #80 percent

(5625, 26)

In [22]:
X_test.shape #20 percent

(1407, 26)

In [23]:
def build_model(input_dim, hidden_layers=(32,), activation='relu', learning_rate=0.001):
    model = Sequential()
    
    # Add Input layer first
    model.add(Input(shape=(input_dim,)))

    # First hidden layer (and others)
    for units in hidden_layers:
        model.add(Dense(units, activation=activation))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


In [24]:
param_grid = {'hidden_layers': [(16,), (32,), (32,16)],
              'activation': ['relu', 'tanh'],
              'learning_rate': [1e-3, 1e-4]
              }

In [37]:
results = []

for hl, act, lr in itertools.product(param_grid['hidden_layers'],param_grid['activation'],param_grid['learning_rate'],):
    print(f"Training model with hidden_layers={hl}, activation={act}, learning_rate={lr}")

    model = build_model(input_dim=X_train.shape[1], hidden_layers=hl, activation=act, learning_rate=lr)

    # Training & time it
    start = time.time()

    es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

    history = model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0, validation_split=0.1,callbacks=[es])
    elapsed = time.time() - start

    # Evaluate
    train_acc = history.history['accuracy'][-1]
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

    results.append({'hidden_layers': hl,'activation': act,'learning_rate': lr,'train_accuracy': train_acc,'test_accuracy': test_acc,'train_time_s': elapsed})
    
print("Complete")

Training model with hidden_layers=(16,), activation=relu, learning_rate=0.001
Restoring model weights from the end of the best epoch: 7.
Epoch 10: early stopping
Training model with hidden_layers=(16,), activation=relu, learning_rate=0.0001
Training model with hidden_layers=(16,), activation=tanh, learning_rate=0.001
Restoring model weights from the end of the best epoch: 12.
Epoch 15: early stopping
Training model with hidden_layers=(16,), activation=tanh, learning_rate=0.0001
Training model with hidden_layers=(32,), activation=relu, learning_rate=0.001
Restoring model weights from the end of the best epoch: 4.
Epoch 7: early stopping
Training model with hidden_layers=(32,), activation=relu, learning_rate=0.0001
Restoring model weights from the end of the best epoch: 41.
Epoch 44: early stopping
Training model with hidden_layers=(32,), activation=tanh, learning_rate=0.001
Restoring model weights from the end of the best epoch: 7.
Epoch 10: early stopping
Training model with hidden_lay

In [38]:
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

hidden_layers activation  learning_rate  train_accuracy  test_accuracy  train_time_s
        (16,)       relu         0.0010        0.806401       0.786780      4.444705
        (16,)       relu         0.0001        0.800474       0.788913     17.412198
        (16,)       tanh         0.0010        0.804623       0.800995      5.620313
        (16,)       tanh         0.0001        0.807388       0.798152     16.561714
        (32,)       relu         0.0010        0.809957       0.791045      3.162086
        (32,)       relu         0.0001        0.799684       0.798152     14.665524
        (32,)       tanh         0.0010        0.805413       0.792466      4.116629
        (32,)       tanh         0.0001        0.807388       0.793888     16.737830
     (32, 16)       relu         0.0010        0.807981       0.786070      3.612670
     (32, 16)       relu         0.0001        0.799881       0.786780      8.252616
     (32, 16)       tanh         0.0010        0.804425       0.7

In [51]:
print("Best Model Configuation")
best = results_df.loc[[results_df['test_accuracy'].idxmax()]]
display(best)

Best Model Configuation


,hidden_layers,activation,learning_rate,train_accuracy,test_accuracy,train_time_s
2,"(16,)",tanh,0.001,0.804623,0.800995,5.620313


In [44]:
# Extract best params
best_params = best.iloc[0]
best_model = build_model(
    input_dim=X_train.shape[1],
    hidden_layers=best_params['hidden_layers'],
    activation=best_params['activation'],
    learning_rate=best_params['learning_rate']
)

best_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

y_pred_prob = best_model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()


44/44 [==============================] - 0s 2ms/step


In [45]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86      1033
           1       0.62      0.55      0.58       374

    accuracy                           0.79      1407
   macro avg       0.73      0.71      0.72      1407
weighted avg       0.78      0.79      0.79      1407



In [47]:
best = results_df.loc[[results_df['test_accuracy'].idxmax()]].iloc[0]

print("Conclusion:")
print(f"- Best hidden layer configuration: {best['hidden_layers']}")
print(f"- Best activation function: {best['activation']}")
print(f"- Best learning rate: {best['learning_rate']}")
print(f"- Training Accuracy: {best['train_accuracy']:.4f}")
print(f"- Test Accuracy: {best['test_accuracy']:.4f}")
print(f"- Training Time: {best['train_time_s']:.2f} seconds")

Conclusion:
- Best hidden layer configuration: (16,)
- Best activation function: tanh
- Best learning rate: 0.001
- Training Accuracy: 0.8046
- Test Accuracy: 0.8010
- Training Time: 5.62 seconds
